In [160]:
from datetime import datetime
import pandas as pd
import locale
import warnings
import os

locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
warnings.filterwarnings("ignore")

In [157]:
def get_file_name() -> list[str]:
    file_list = []
    for file in os.listdir():
        if file.endswith(".xlsx") and "Base Pagos" in file:
            file_list.append(file)
    return file_list

def get_last_date(dates_list: list[datetime]) -> datetime:
    last_date = dates_list[0]
    for date in dates_list:
        if date > last_date:
            last_date = date
    return last_date

def get_date(file_list: list[str]) -> str:
    dates_list = []
    for file in file_list:
        fecha_list =  file.split(" ")[-1].split(".")[:3]
        dates_list.append(datetime.strptime(fecha_list[0] + "." + fecha_list[1] + "." + fecha_list[2], "%d.%m.%Y"))
    return get_last_date(dates_list).strftime("%d.%m.%Y")

fecha = get_date(get_file_name())
fecha

'20.08.2024'

In [168]:
def validar_o_crear_carpeta() -> str:
    fecha_hoy = datetime.today().strftime('%Y.%m.%d')
    
    carpeta_reporte = 'reporte'
    
    carpeta_hoy = os.path.join(carpeta_reporte, fecha_hoy)
    
    if not os.path.exists(carpeta_hoy):
        os.makedirs(carpeta_hoy)
        print(f'Carpeta creada: {carpeta_hoy}')
    else:
        print(f'La carpeta ya existe: {carpeta_hoy}')
    
    return carpeta_hoy

file_path = validar_o_crear_carpeta()
file_path

La carpeta ya existe: reporte\2024.08.21


'reporte\\2024.08.21'

In [177]:
base_pagos_path = "Base Pagos " + fecha + ".xlsx"
asignacion_path = "base_asignacion_20240813.xlsx"

print(base_pagos_path)
print(asignacion_path)

monoproducto = file_path + "/MONOPRODUCTO " + fecha + ".xlsx"
multiproducto = file_path + "/MULTIPRODUCTO " + fecha + ".xlsx"
reactiva = file_path + "/REACTIVA " + fecha + ".xlsx"

mono_path = os.path.abspath(monoproducto)
multi_path = os.path.abspath(multiproducto)
react_path = os.path.abspath(reactiva)

print(mono_path)
print(multi_path)
print(react_path)

Base Pagos 20.08.2024.xlsx
base_asignacion_20240813.xlsx
c:\Users\miria\Documents\Mirko\BBVA\GitHub\base-pagos\reporte\2024.08.21\MONOPRODUCTO 20.08.2024.xlsx
c:\Users\miria\Documents\Mirko\BBVA\GitHub\base-pagos\reporte\2024.08.21\MULTIPRODUCTO 20.08.2024.xlsx
c:\Users\miria\Documents\Mirko\BBVA\GitHub\base-pagos\reporte\2024.08.21\REACTIVA 20.08.2024.xlsx


In [54]:
#################### BASE DE PAGOS ####################

In [162]:
df_base = pd.read_excel(base_pagos_path)
print(df_base.shape)
print(df_base.dtypes)
df_base.head(5)

(226, 9)
Entidad         int64
FECHA          object
CODCEN          int64
CLAVSERV        int64
CENTROPAGO      int64
IMPORTE       float64
MONEDA         object
NOMBRE         object
ESTADO         object
dtype: object


,Entidad,FECHA,CODCEN,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO
0,11,2024-08-20,21497357,248,341,10000.0,PEN,"A CUENTA DE PRESTAMO,",P
1,11,2024-08-20,24674746,248,229,2000.0,PEN,AGRONEGOCIOS CHELA SCRL,P
2,11,2024-08-20,24761696,248,564,700.0,PEN,AGUIDA HONORIA PACHECO LEON,P
3,11,2024-08-20,26742581,248,275,1939.0,PEN,ALBI ELVI OLORTEGUI JIMENEZ,P
4,11,2024-08-20,29039532,248,849,62.0,PEN,ALESSANDRA MILAGROS IRAZABAL FLORES,P


In [109]:
fecha_formateada = pd.to_datetime('today').strftime('%d-%b')
fecha_formateada = fecha_formateada[:3] + fecha_formateada[3:].capitalize()

df_base['FECHA_ENVIO'] = fecha_formateada.replace('.', '')
df_base['ID_RESPONSABLE'] = 'MIV'

cols_base = ['FECHA', 'CODCEN', 'CLAVSERV', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FECHA_ENVIO', 'ID_RESPONSABLE']
df_base = df_base[cols_base]
df_base = df_base.rename(columns={'CODCEN': 'CC'})

df_base['CC'] = df_base['CC'].astype("Int64").astype(str).str.zfill(8)
df_base['CC'] = df_base['CC'].str.replace(' ', '').str.replace(r'\D', '', regex=True).str[-8:]
df_base['CC'] = df_base['CC'].str.zfill(8)

df_base['CLAVSERV'] = df_base["CLAVSERV"].astype(str).str.zfill(5)

print(df_base.shape)
print(df_base.dtypes)
df_base.head(5)

(226, 9)
FECHA              object
CC                 object
CLAVSERV           object
IMPORTE           float64
MONEDA             object
NOMBRE             object
ESTADO             object
FECHA_ENVIO        object
ID_RESPONSABLE     object
dtype: object


,FECHA,CC,CLAVSERV,IMPORTE,MONEDA,NOMBRE,ESTADO,FECHA_ENVIO,ID_RESPONSABLE
0,2024-08-20,21497357,00248,10000.0,PEN,"A CUENTA DE PRESTAMO,",P,21-Ago,MIV
1,2024-08-20,24674746,00248,2000.0,PEN,AGRONEGOCIOS CHELA SCRL,P,21-Ago,MIV
2,2024-08-20,24761696,00248,700.0,PEN,AGUIDA HONORIA PACHECO LEON,P,21-Ago,MIV
3,2024-08-20,26742581,00248,1939.0,PEN,ALBI ELVI OLORTEGUI JIMENEZ,P,21-Ago,MIV
4,2024-08-20,29039532,00248,62.0,PEN,ALESSANDRA MILAGROS IRAZABAL FLORES,P,21-Ago,MIV


In [110]:
df_base.isnull().sum()

FECHA             0
CC                0
CLAVSERV          0
IMPORTE           0
MONEDA            0
NOMBRE            0
ESTADO            0
FECHA_ENVIO       0
ID_RESPONSABLE    0
dtype: int64

In [7]:
#################### ASIGNACION ####################

In [8]:
df_asignacion = pd.read_excel(asignacion_path)
print(df_asignacion.shape)
print(df_asignacion.dtypes)
df_asignacion.head(5)

(242511, 40)
codigo                        int64
contrato                      int64
cont_18                       int64
nombre                       object
clave                        object
producto                     object
DiasVenc                      int64
moneda                       object
capital                     float64
liqtotal                    float64
agencia                      object
Camp                        float64
Gestor                       object
numdoc                       object
oficina                      object
territorio                   object
Tramo                        object
codoficina                    int64
CAPITAL_SOLES               float64
IMP_CAMPAÑA                 float64
TIPO_FONDO                   object
FLAG_HONRADO                  int64
TIPO_CARTERA                 object
DESC_TRAMO                   object
Hibrido                      object
ESTUDIO ABOGADO              object
FECHA_MORA                   object
ID_CASTIGO     

,codigo,contrato,cont_18,nombre,clave,producto,DiasVenc,moneda,capital,liqtotal,...,ID_CONDONACION MORA_TEMP,ID_FRACCIONADO,ID_KOBSA_P.CANCELACION,ID_REPORTE_CANC,ID_CARTERA,ID_FLAG,OBSERVACION,AGENCIA CORRECTA,PAG LEGAL,PAG. INTER
0,30739481,110010629600055161,1100109600055161,DINA CHOQUE FERNANDEZ,BC,PRÉSTAMOS PYMES,257,PEN,76663.97,108418.69,...,-,-,-,-,STOCK,2,PAGO LEGAL AGOSTO // SE CORRIGIO,MORNESE MORA,1,-
1,30739481,110010619600057679,1100109600057679,DINA CHOQUE FERNANDEZ,BC,PRÉSTAMOS PYMES,268,PEN,177075.30,256680.65,...,-,-,-,-,STOCK,2,PAGO LEGAL AGOSTO // SE CORRIGIO,MORNESE MORA,1,-
2,21874900,110050295000360222,1100505000360222,JULIA ANGELICA SOTO COTRINA,BC,TARJETAS PYMES,307,PEN,23986.04,36966.81,...,-,-,-,-,STOCK,4,PAGO INTERBANCARIO // SE CORRIGIO,ASESCOM RJ,-,1
3,30293392,110130235002987960,1101305002987960,INNOVA FASHION GROUPSAC,BC,TARJETAS PYMES,215,PEN,142100.58,200698.07,...,-,-,-,-,STOCK,1,PAGO INTERBANCARIO // SE CORRIGIO,CLASA MORA,-,1
4,20687681,110174079600365104,1101749600365104,ELIZABETH HAYDEE CORDOVA AMAYA,BC,PRÉSTAMOS PYMES,356,PEN,200151.61,309435.56,...,-,1,-,-,STOCK,1,MARCA FRACCIONADO RETORNAR AGENCIA // SE CORRIGIO,RodriguezAngobaldo,-,-


In [112]:
#################### COPIAS ####################

In [113]:
df_base_test = df_base.copy()
print("Base", df_base_test.shape)

df_asignacion_test = df_asignacion.copy()
print("Asignacion", df_asignacion_test.shape)

Base (226, 9)
Asignacion (242511, 40)


In [114]:
df_asignacion_test['codigo'] = df_asignacion_test['codigo'].astype("Int64").astype(str).str.zfill(8)

cols_asignacion = ['codigo', 'ID_FLAG', 'cont_18', 'nombre', 'TIPO_CARTERA', 'TIPO_FONDO', 'clave', 'AGENCIA CORRECTA']
df_base_test = df_base_test.merge(df_asignacion_test[cols_asignacion], left_on='CC', right_on='codigo', how='left')
df_base_test = df_base_test.drop(columns=['codigo'])
df_base_test = df_base_test.rename(columns={'ID_FLAG': 'FLAG', 'cont_18': 'CONTRATO', 'nombre': 'NOMBRE_CLIENTE', 'clave': 'CARTERA', 'AGENCIA CORRECTA': 'AGENCIA'})

df_base_test["TIPO_CARTERA"] = df_base_test["TIPO_CARTERA"].fillna("NULL")
df_base_test["TIPO_FONDO"] = df_base_test["TIPO_FONDO"].fillna("NULL")

df_base_test["FLAG"] = df_base_test["FLAG"].astype("Int64")
df_base_test["CONTRATO"] = df_base_test["CONTRATO"].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)

cols_base_test = ['FECHA', 'CC', 'CLAVSERV', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'CONTRATO', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'TIPO_FONDO', 'CARTERA', 'AGENCIA']
df_base_test = df_base_test[cols_base_test]

print(df_base_test.shape)
print(df_base_test.dtypes)
df_base_test.head(5)

(279, 16)
FECHA              object
CC                 object
CLAVSERV           object
IMPORTE           float64
MONEDA             object
NOMBRE             object
ESTADO             object
FLAG                Int64
CONTRATO           object
NOMBRE_CLIENTE     object
FECHA_ENVIO        object
ID_RESPONSABLE     object
TIPO_CARTERA       object
TIPO_FONDO         object
CARTERA            object
AGENCIA            object
dtype: object


,FECHA,CC,CLAVSERV,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,TIPO_FONDO,CARTERA,AGENCIA
0,2024-08-20,21497357,00248,10000.0,PEN,"A CUENTA DE PRESTAMO,",P,2,001103419600130465,EMPRESA DE TRANSPORTES NEGOCIOS E IMPORTACIONE...,21-Ago,MIV,UNSECURED,NULL,BC,ASESCOM RJ
1,2024-08-20,21497357,00248,10000.0,PEN,"A CUENTA DE PRESTAMO,",P,2,001103415001339075,EMPRESA DE TRANSPORTES NEGOCIOS E IMPORTACIONE...,21-Ago,MIV,UNSECURED,NULL,BC,ASESCOM RJ
2,2024-08-20,24674746,00248,2000.0,PEN,AGRONEGOCIOS CHELA SCRL,P,2,001102299600419261,AGRONEGOCIOS CHELA SCRL,21-Ago,MIV,UNSECURED,REACTIVA,KSTBC,MORNESE MORA
3,2024-08-20,24674746,00248,2000.0,PEN,AGRONEGOCIOS CHELA SCRL,P,2,001102295001867072,AGRONEGOCIOS CHELA SCRL,21-Ago,MIV,UNSECURED,NULL,KSTBC,MORNESE MORA
4,2024-08-20,24761696,00248,700.0,PEN,AGUIDA HONORIA PACHECO LEON,P,1,001107859600460400,AGUIDA HONORIAPACHECOLEON,21-Ago,MIV,UNSECURED,NULL,EXJ,MORNESE MORA


In [115]:
#################### MONOPRODUCTO ####################

In [116]:
df_mono = df_base_test[df_base_test["FLAG"] == 1]
print(df_mono.shape)
df_mono.head(5)

(164, 16)


,FECHA,CC,CLAVSERV,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,TIPO_FONDO,CARTERA,AGENCIA
4,2024-08-20,24761696,00248,700.0,PEN,AGUIDA HONORIA PACHECO LEON,P,1,001107859600460400,AGUIDA HONORIAPACHECOLEON,21-Ago,MIV,UNSECURED,NULL,EXJ,MORNESE MORA
5,2024-08-20,26742581,00248,1939.0,PEN,ALBI ELVI OLORTEGUI JIMENEZ,P,1,001102769600217452,ALBI ELVI OLORTEGUI JIMENEZ,21-Ago,MIV,UNSECURED,NULL,EXJ,ASESCOM RJ
6,2024-08-20,29039532,00248,62.0,PEN,ALESSANDRA MILAGROS IRAZABAL FLORES,P,1,001108495000622220,ALESSANDRA MILAGROS IRAZABAL FLORES,21-Ago,MIV,UNSECURED,NULL,KSTBC,MORNESE MORA
7,2024-08-20,24210837,00248,100.0,PEN,ALEX EDUARDO SIVIRICHI AYACHI,P,1,001104085012434340,ALEX EDUARDO SIVIRICHI AYACHI,21-Ago,MIV,UNSECURED,NULL,KSTBC,MORNESE MORA
10,2024-08-20,25425006,00248,1400.0,PEN,ALEX VASQUEZ MATTA,P,1,001102729600053539,ALEX VASQUEZ MATTA,21-Ago,MIV,UNSECURED,NULL,EXJ,MORNESE MORA


In [117]:
df_mono["TIPO_CARTERA"].value_counts()

TIPO_CARTERA
UNSECURED    152
SECURED       12
Name: count, dtype: int64

In [118]:
df_mono["TIPO_FONDO"].value_counts()

TIPO_FONDO
NULL        162
REACTIVA      2
Name: count, dtype: int64

In [119]:
df_mono[["TIPO_CARTERA", "TIPO_FONDO"]].value_counts()

TIPO_CARTERA  TIPO_FONDO
UNSECURED     NULL          151
SECURED       NULL           11
              REACTIVA        1
UNSECURED     REACTIVA        1
Name: count, dtype: int64

In [17]:
#################### MONO REACTIVA ####################

In [120]:
df_reactiva = df_mono[(df_mono["TIPO_CARTERA"] == "UNSECURED") & (df_mono["TIPO_FONDO"] == "REACTIVA")]
print(df_reactiva.shape)
df_reactiva.head(5)

(1, 16)


,FECHA,CC,CLAVSERV,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,TIPO_FONDO,CARTERA,AGENCIA
182,2024-08-20,28350038,00248,1500.0,PEN,MULTISERVICIOS Z & M EIRL,P,1,001104849600201468,MULTISERVICIOS Z&M EIRL,21-Ago,MIV,UNSECURED,REACTIVA,BC,Muñiz


In [173]:
df_reactiva.to_excel(reactiva, index=False)
#os.startfile(os.path.abspath(reactiva))

In [122]:
#################### FIN REACTIVA ####################

In [123]:
df_mono = df_mono[df_mono["TIPO_CARTERA"] == "UNSECURED"]
df_mono = df_mono[df_mono["TIPO_FONDO"] == "NULL"]
print(df_mono.shape)

(151, 16)


In [124]:
df_mono["CONTRATO"] = df_mono["CONTRATO"].astype(str).str.zfill(18)

In [172]:
df_mono.to_excel(monoproducto, index=False)
#os.startfile(os.path.abspath(monoproducto))

In [24]:
#################### FIN MONOPRODUCTO ####################

In [25]:
#################### MULTIPRODUCTO ####################

In [126]:
df_multi = df_base_test[df_base_test["FLAG"] > 1].copy()
print(df_multi.shape)
df_multi.head(5)

(102, 16)


,FECHA,CC,CLAVSERV,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,TIPO_FONDO,CARTERA,AGENCIA
0,2024-08-20,21497357,00248,10000.0,PEN,"A CUENTA DE PRESTAMO,",P,2,001103419600130465,EMPRESA DE TRANSPORTES NEGOCIOS E IMPORTACIONE...,21-Ago,MIV,UNSECURED,NULL,BC,ASESCOM RJ
1,2024-08-20,21497357,00248,10000.0,PEN,"A CUENTA DE PRESTAMO,",P,2,001103415001339075,EMPRESA DE TRANSPORTES NEGOCIOS E IMPORTACIONE...,21-Ago,MIV,UNSECURED,NULL,BC,ASESCOM RJ
2,2024-08-20,24674746,00248,2000.0,PEN,AGRONEGOCIOS CHELA SCRL,P,2,001102299600419261,AGRONEGOCIOS CHELA SCRL,21-Ago,MIV,UNSECURED,REACTIVA,KSTBC,MORNESE MORA
3,2024-08-20,24674746,00248,2000.0,PEN,AGRONEGOCIOS CHELA SCRL,P,2,001102295001867072,AGRONEGOCIOS CHELA SCRL,21-Ago,MIV,UNSECURED,NULL,KSTBC,MORNESE MORA
8,2024-08-20,27301901,00248,250.0,PEN,ALEX BERROCAL PINEDO,P,2,001103169600200939,ALEX BERROCAL PINEDO,21-Ago,MIV,UNSECURED,NULL,BC,CLASA MORA


In [127]:
df_multi = df_multi[df_multi["TIPO_CARTERA"] == "UNSECURED"]

df_multi["CONTRATO"] = None
df_multi["TIPO_FONDO"] = None
df_multi["CARTERA"] = None

df_multi = df_multi.drop(columns=['TIPO_CARTERA', 'AGENCIA'])
df_multi = df_multi.drop_duplicates(subset=['CC', 'IMPORTE', 'MONEDA'])

cols_multi = ['FECHA', 'CC', 'CLAVSERV', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE']
df_multi = df_multi[cols_multi]

print(df_multi.shape)
df_multi.head(5)

(46, 14)


,FECHA,CC,CLAVSERV,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE
0,2024-08-20,21497357,00248,10000.0,PEN,"A CUENTA DE PRESTAMO,",P,2,None,None,None,EMPRESA DE TRANSPORTES NEGOCIOS E IMPORTACIONE...,21-Ago,MIV
2,2024-08-20,24674746,00248,2000.0,PEN,AGRONEGOCIOS CHELA SCRL,P,2,None,None,None,AGRONEGOCIOS CHELA SCRL,21-Ago,MIV
8,2024-08-20,27301901,00248,250.0,PEN,ALEX BERROCAL PINEDO,P,2,None,None,None,ALEX BERROCAL PINEDO,21-Ago,MIV
12,2024-08-20,30474884,00248,327.0,PEN,ALEXIS HERRERA MORE,P,2,None,None,None,ALEXIS HERRERA MORE,21-Ago,MIV
16,2024-08-20,24318792,00248,500.0,PEN,ALIAGA LOZANO GODOFREDO,P,2,None,None,None,GODOFREDO ALIAGA LOZANO,21-Ago,MIV


In [171]:
df_multi.to_excel(multiproducto, index=False)
#os.startfile(os.path.abspath(multiproducto))

In [129]:
#################### FIN MULTIPRODUCTO ####################

In [169]:
import openpyxl
from openpyxl.styles import Font, PatternFill

def format_excel(file_path: str, validator: str) -> None:
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook.active
    
    # Definir estilos
    header_font = Font(bold=True, color="FFFFFF")
    header_fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")
    
    # Aplicar estilos a la primera fila (asumiendo que es el encabezado)
    for cell in sheet[1]:
        cell.font = header_font
        cell.fill = header_fill
    
    # Aplicar estilos específicos según el validador
    if validator == "mono":
        for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row, min_col=1, max_col=sheet.max_column):
            for cell in row:
                if cell.value == "Monoproducto":
                    cell.font = Font(color="FF0000")
    elif validator == "multi":
        for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row, min_col=1, max_col=sheet.max_column):
            for cell in row:
                if cell.value == "Multiproducto":
                    cell.font = Font(color="00FF00")
    elif validator == "react":
        for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row, min_col=1, max_col=sheet.max_column):
            for cell in row:
                if cell.value == "Reactiva":
                    cell.font = Font(color="0000FF")
    
    # Guardar el archivo Excel
    workbook.save(file_path)

In [174]:
format_excel(mono_path, "mono")
format_excel(multi_path, "multi")
format_excel(react_path, "react")

os.startfile(mono_path)
os.startfile(multi_path)
os.startfile(react_path)